## DATA 602 Fall 2024 - Assignment 7
### Stephanie Chiang

## NYC SHSAT

For this assignment, I will examine the NYC middle schools and the numbers of students who "participated in High School Admissions, the number of those students who took the Specialized High Schools Admissions Test (SHSAT) and the number who received an offer to one of the 8 testing Specialized High Schools" for the 2020-2021 school year.

The raw file for the *2020-2021 SHSAT Admissions Test Offers By Sending School*, provided by the NYC Department of Education, is available for download [here]("https://data.cityofnewyork.us/Education/2020-2021-SHSAT-Admissions-Test-Offers-By-Sending-/k8ah-28f4/about_data").


### Data Exploration

- importing the dataset and creating dataframes
- missing value information
- relevant information about the dataset
- summary statistics means, medians, quartiles

In [142]:
import pandas as pd

data = pd.read_csv("2021shsat.csv")

# modifying multiple column names to more code-friendly format
data.rename(
  columns = {
    "Feeder School DBN": "DBN",
    "Feeder School Name": "school",
    "Count of Students in HS Admissions": "hs_bound",
    "Count of Testers": "testers",
    "Number of Offers": "offers"
  },
  inplace = True
)

print(data.isna().sum())
# there appears to be no missing data

print(data.shape)
# there are 658 rows (observations) and 5 columns:
# the middle school's District/Borough Number
# the name of the middle school
# the number of students who participated in HS admissions
# the number of students who took the SHSAT
# and the number of students who received an offer from a specialized HS

print(data.dtypes)
# all the columns are listed as 'object' which indicates mixed types or categorical features
# this is because '0-5' appears as a (very common) possible value
# so pandas was unable to automatically convert these columns to integer types

DBN         0
school      0
hs_bound    0
testers     0
offers      0
dtype: int64
(658, 5)
DBN         object
school      object
hs_bound    object
testers     object
offers      object
dtype: object



The last 3 columns include values of "0-5" for any count of 5 students or fewer. There are a few different ways this could be handled, but for the sake of calculating summary statistics, I will convert these columns to floats, with a mean of 2.5 for the "0-5" values. Since there are no half-students counted in the original data, this means they can still be grouped easily from the rest of the data.

In [143]:
# convert variables to proper types

data["hs_bound"] = data["hs_bound"].where(data["hs_bound"] != "0-5", "2.5")
data["testers"] = data["testers"].where(data["testers"] != "0-5", "2.5")
data["offers"] = data["offers"].where(data["offers"] != "0-5", "2.5")

data[["hs_bound", "testers", "offers"]] = data[["hs_bound", "testers", "offers"]].astype(float)

print(data.dtypes)
print(data.head(5))

# summary statistics: mean, median and quantile
print(data.describe())

DBN          object
school       object
hs_bound    float64
testers     float64
offers      float64
dtype: object
      DBN                                          school  hs_bound  testers  \
0  01M034         P.S. 034 FRANKLIN D. ROOSEVELT (01M034)      44.0      2.5   
1  01M140                 P.S. 140 NATHAN STRAUS (01M140)      56.0      9.0   
2  01M184                   P.S. 184M SHUANG WEN (01M184)     112.0     79.0   
3  01M188             P.S. 188 THE ISLAND SCHOOL (01M188)      49.0      2.5   
4  01M332  UNIVERSITY NEIGHBORHOOD MIDDLE SCHOOL (01M332)      70.0     10.0   

   offers  
0     2.5  
1     2.5  
2    29.0  
3     2.5  
4     2.5  


### DATA WRANGLING

In [145]:
# Create new columns based on existing columns or calculations.
data["pct_testers"] = data["testers"] / data["hs_bound"]
data["pct_offers"] = data["offers"] / data["hs_bound"]

print(data.head(5))

# Drop column(s) from your dataset.
data = data.drop(["DBN"], axis=1)

# Drop a row(s) from your dataset.
idx_max = data['hs_bound'].idxmax()
data_dropped_max_hs = data.drop([idx_max])

print(data.shape) # there is now one less row and one less column

      DBN                                          school  hs_bound  testers  \
0  01M034         P.S. 034 FRANKLIN D. ROOSEVELT (01M034)      44.0      2.5   
1  01M140                 P.S. 140 NATHAN STRAUS (01M140)      56.0      9.0   
2  01M184                   P.S. 184M SHUANG WEN (01M184)     112.0     79.0   
3  01M188             P.S. 188 THE ISLAND SCHOOL (01M188)      49.0      2.5   
4  01M332  UNIVERSITY NEIGHBORHOOD MIDDLE SCHOOL (01M332)      70.0     10.0   

   offers  pct_testers  pct_offers  
0     2.5     0.056818    0.056818  
1     2.5     0.160714    0.044643  
2    29.0     0.705357    0.258929  
3     2.5     0.051020    0.051020  
4     2.5     0.142857    0.035714  
(657, 6)


In [147]:

# Filter your data based on some condition.
ignore_small_vals = data[data["testers"] != 2.5]

print(ignore_small_vals.head(3))

# Sort your data based on multiple variables.
ignore_small_vals = ignore_small_vals.sort_values(by=['pct_offers', 'school'], ascending=[False, True])

print(ignore_small_vals.head(3))

# Convert all the string values to upper or lower cases in one column.
data_lower = data["school"].str.lower()

print(data_lower.head(3))


                                           school  hs_bound  testers  offers  \
1                 P.S. 140 NATHAN STRAUS (01M140)      56.0      9.0     2.5   
2                   P.S. 184M SHUANG WEN (01M184)     112.0     79.0    29.0   
4  UNIVERSITY NEIGHBORHOOD MIDDLE SCHOOL (01M332)      70.0     10.0     2.5   

   pct_testers  pct_offers  
1     0.160714    0.044643  
2     0.705357    0.258929  
4     0.142857    0.035714  
                                                school  hs_bound  testers  \
45                        THE ANDERSON SCHOOL (03M334)      60.0     55.0   
22   NEW YORK CITY LAB MIDDLE SCHOOL FOR COLLABORAT...     179.0    169.0   
294     THE CHRISTA MCAULIFFE SCHOOL\I.S. 187 (20K187)     340.0    315.0   

     offers  pct_testers  pct_offers  
45     45.0     0.916667    0.750000  
22    117.0     0.944134    0.653631  
294   221.0     0.926471    0.650000  
0    p.s. 034 franklin d. roosevelt (01m034)
1            p.s. 140 nathan straus (01m140)
2       

In [152]:

# Group your dataset by one column, and get the mean, min, and max values by group.
# .groupby() .agg() or .apply()

grouped_data = data.groupby("offers").agg({
  "hs_bound": ["mean", "min", "max"],
  "testers": ["mean", "min", "max"]
})

print(grouped_data.head(15))

          hs_bound                   testers              
              mean    min    max        mean    min    max
offers                                                    
2.5      79.878205    2.5  708.0   16.954212    2.5  164.0
6.0     224.700000   49.0  495.0   87.300000   22.0  169.0
7.0     219.600000   87.0  585.0   81.400000   38.0  159.0
8.0     167.500000   60.0  368.0   55.833333   26.0  117.0
9.0     121.444444   54.0  340.0   46.222222   28.0   93.0
10.0    206.200000  104.0  549.0   75.400000   47.0  157.0
11.0    199.666667   92.0  358.0   65.000000   53.0   77.0
12.0    187.333333  105.0  316.0   66.000000   29.0   96.0
13.0    519.000000  428.0  610.0  127.500000  123.0  132.0
14.0    125.500000   59.0  307.0   54.000000   32.0   76.0
15.0    104.500000   99.0  110.0   42.500000   20.0   65.0
16.0    328.000000  151.0  505.0   99.500000   65.0  134.0
17.0    197.200000   65.0  524.0   80.600000   49.0  153.0
18.0    315.000000   85.0  516.0  123.400000   52.0  164

In [156]:
# Group your dataset by two columns and then sort the aggregated results within the groups.

grouped_data_2 = ignore_small_vals.groupby(["offers", "pct_offers"]).agg({
  "testers": ["mean", "min", "max"],
  "pct_testers": ["mean", "min", "max"]
})

grouped_data_2 = grouped_data_2.sort_values(('testers', 'mean'), ascending=False)

print(grouped_data_2.head(15))

                  testers               pct_testers                    
                     mean    min    max        mean       min       max
offers pct_offers                                                      
110.0  0.205993     374.0  374.0  374.0    0.700375  0.700375  0.700375
102.0  0.178634     342.0  342.0  342.0    0.598949  0.598949  0.598949
82.0   0.141869     318.0  318.0  318.0    0.550173  0.550173  0.550173
221.0  0.650000     315.0  315.0  315.0    0.926471  0.926471  0.926471
151.0  0.356132     298.0  298.0  298.0    0.702830  0.702830  0.702830
64.0   0.120983     259.0  259.0  259.0    0.489603  0.489603  0.489603
80.0   0.167364     248.0  248.0  248.0    0.518828  0.518828  0.518828
78.0   0.214286     233.0  233.0  233.0    0.640110  0.640110  0.640110
73.0   0.173810     227.0  227.0  227.0    0.540476  0.540476  0.540476
82.0   0.200980     221.0  221.0  221.0    0.541667  0.541667  0.541667
126.0  0.480916     219.0  219.0  219.0    0.835878  0.835878  0


### CONCLUSIONS
The school with the highest precentage of high school boound middle schoolers receiving an offer was 'THE ANDERSON SCHOOL'. However, based on how the most common number of offers made was '0-5', it could be assumed that most schools did not have any students receive any offers.

Given more time and better data, I would like to figure out why the data was listed in this way - excluding values 5 and leads to biased results and distorts the summary statistics. This may lead people to draw the wrong conclusions from the data.